In [6]:
import numpy as np
import pandas as pd
import graphlab
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

BASE  = './bytecup2016data'
IINFO = BASE + '/invited_info_train.txt'
QINFO = BASE + '/question_info.txt'
VAL   = BASE + '/validate_nolabel.txt'
TEST   = BASE + '/test_nolabel.txt'

invdata = pd.read_csv(IINFO, delim_whitespace=True, header=None, names=["qid", "uid", "label"])
qdata   = pd.read_csv(QINFO, delim_whitespace=True, header=None, names=["qid", "qtag", "wseq", "cseq", "nvotes", "nans", "ntqans"])
valdata = pd.read_csv(TEST)

qdata = qdata.drop(["wseq","cseq","qtag"], axis = 1)

In [7]:
def algorithm_predict(new_invdata, new_valdata, qdata):
    sf = graphlab.SFrame(new_invdata, format='dataframe')
    #side_item = graphlab.SFrame(qdata, format='dataframe')
    #m = graphlab.recommender.create(sf, target='rating')
    #m = graphlab.item_similarity_recommender.create(sf, user_id='uid', item_id='qid', target='label', similarity_type='pearson')
    #m = graphlab.factorization_recommender.create(sf, target='label', regularization=0.0001, num_factors = 32,
                                                  #max_iterations = 25, linear_regularization = 1e-05, user_id='uid', item_id='qid')
    
    m = graphlab.recommender.ranking_factorization_recommender.create(sf, target='label', regularization=1e-08, num_factors = 32,
                                                  max_iterations = 25, ranking_regularization = 0.1, user_id='uid', item_id='qid',
                                                                     num_sampled_negative_examples = 8)
    #m = graphlab.popularity_recommender.create(sf, target='rating')  
    
    sf = graphlab.SFrame(new_valdata, format='dataframe')
    return m.predict(sf)

In [8]:
valdata['label'] = algorithm_predict(invdata, valdata, qdata)
valdata.to_csv('matrix_factorization-recommender.csv', separator=",")

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\RAJBHA~1\AppData\Local\Temp\graphlab_server_1481547220.log.0


This non-commercial license of GraphLab Create for academic use is assigned to rajbharr@usc.edu and will expire on November 18, 2017.


Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 245752 observations with 27127 users and 7708 items.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.0848499

Final training RMSE: 0.222373